# Molecular dating of Holocephali using fossil calibrations


## 1. Introduction

### 1.1 L'article
Grâce à la leur position à la base des poissons __osseux__, les poissons __cartilagineux__ sont considérés comme un excellent _outgroup_ dans les études comparatives des vertebrés.
<br/>
Récemment, le génome de _Callorhinchus milii_ (__chimère éléphant__) a été proposé comme génome modèle pour comprendre l'histoire évolutive des génomes vertebrés grâce sa position phylogénétique et à son génome de petite taille (910 Mb). Malgré un tel intérêt croissant, l’histoire évolutive des _holophaceans_ modernes ainsi que leur rélation avec _Elasmobranchii_ (sous-classe de _Chondrichthyes_, qui regroupe les requins, raies et chimères) et d'autres vertebrés à machoire n'a pas été bien documentée à cause de l'absence de matériaux fossiles bien conservés.
<br/><br/>
Dans cette étude, les auteurs ont assemblé le génome mitchondrial de 8 représentants de trois familles d'_holophaceans_ modernes, pour pouvoir étudier leurs rélations phylogénétiques et histoire évolutive. Pour faire cela, ils ont analysé par __maximum__ de __vraisemblance__ les séquences alignées sans ambiguïté de ces _holophaceans_ avec 17 autres vertebrés (9,409 positions nucléotidiques en excludant la troisième position du codon).
<br/>
L'arbre généré supporte l'hypothèse d'une seule origine des _holophaceans_ modernes et leur relation avec le groupe-soeur des _Elasmobranchii_. 
L’arbre mitogénomique a récupéré le plus grand nombre de _Callorhinchus_ basaux dans les _Chimaeriformes_, lequel est groupe-soeur avec les clades des deux familles restantes (_Rhinochimaeridae_ et _Chimaeridae_). 
<br/>
L’arbre temporel dérivé d’une horloge moléculaire détendue bayésienne suggère que les _holophaceans_ sont originaires du __Silurien__ vers 420 Ma, ayant survécu dans la fin du __Permien__ à l'extinction de masse et qui ont subi une diversification familiale de la fin du __Jurassique__ au début du __Crétacé__. Ce scénario évolutif proposé est bien en accord avec celui basé sur les observations paléontologiques.
<br/><br/>
Source: [Evolutionary Origin and Phylogeny of the Modern Holocephalans (Chondrichthyes: Chimaeriformes): A Mitogenomic Perspective](https://academic.oup.com/mbe/article/27/11/2576/1121974?login=true)

### 1.2 Le projet
La datation d’une phylogénie repose sur l’interprétation de la divergence entre les séquences à la lumière des calibrations fossiles, et à la lumière des _a priori_ sur ces calibrations et sur la forme de la phylogénie. <br/>
Il s’agit d’un problème inférentiel difficile où les _a priori_ et les __mouvements__ qui sont utilisés peuvent avoir un impact important sur le résultat et sur la convergence du __MCMC__. Dans ce projet, on va ré-analyser un ensemble de données construit par _Inoue et al._ (cfr. __1.1 L'article__) pour évaluer la robustesse de leurs résultats par rapport aux _a priori_ ou aux __mouvements__ qui sont utilisés.

### 1.3 Les données
Comme anticipé dans la section __1.1 L'article__, les données sont constituées par des génomes mitochrondiaux entiers assemblés de 8 _holophaceans_ (répresentant les trois familles):
- _Callorhinchus milii_ (**_Callorhinchidae_**)
- _Callorhinchus capensis_ (**_Callorhinchidae_**)
- _Callorhinchus callorynchus_ (**_Callorhinchidae_**)
- _Hariotta raleighana_ (**_Rhinochimaeridae_**)
- _Rhinochimaera pacifica_ (**_Rhinochimaeridae_**)
- _Hydrolagus lemures_ (**_Chimaeridae_**)
- _Chimaera sp_ (**_Chimaeridae_**)
- _Chimaera monstrosa_ (**_Chimaeridae_**)

Et 15 autres vertebres: _Cyprinus carpio_, _Polymixia japonica_, _Tetraodon nigroviridis_, _Protopterus aethiopicus_, _Xenopus tropicalis_, _Struthio camelus_, _Ornithorhynchus anatinus_, _Homo sapiens_, _Plesiobatis daviesi_, _Okamejei kenojei_, _Amblyraja radiata_, _Squalus acanthias_, _Heterodontus francisci_, _Scyliorhinus canicula_, _Mustelus manazo_.
<br/>
Les séquences sont contenues dans le fichier `aln_prot.fst`. Il contient les séquences d'acides aminés de 12 gènes codant des protéines des génomes mitochondriaux (exclusion des gènes ARNt et ARNr).

- Origin time: $\phi \sim Unif(300, 750)$
<!-- - Fossilization rate: $\psi \sim Exp(10)$ -->
- Speciation rate: $\lambda \approx 1$
- Sampling rate: $\rho = 0$
- Exctintion rate: $\mu = 1$
- Topology and divergence time: $\tau$
<br/>

- Substitution matrix: $Q$ (MtRev model)
<br/>

- clockrate mean: $m \sim Exp(1)$
- clock rate relative variance: $v \sim Exp(1)$
- alpha _Gamma_'s parameter: $1 / v$
- beta _Gamma_'s parameter: $\alpha / m$
- clock rate: $c \sim Gamma(\alpha, \beta)$

In [1]:
################################################################################
#
# RevBayes script: BDP analysis with fossil calibrations
#
# authors: Barberis Tommaso, Verneret Marie, Chareyre Marie, Frouté Timothée
#
################################################################################

#######################
# Reading in the Data #
#######################
# read nucleotide data
data <- readDiscreteCharacterData("../Projet/fishes/aln_prot.fst")

# n_taxa <- taxa.size()
taxa <- data.taxa()
n_species <- data.ntaxa()
n_branches <- 2 * n_species - 2

moves = VectorMoves()

   Missing Variable:	Variable os does not exist

   Successfully read one character matrix from file '../Projet/fishes/aln_prot.fst'


In [2]:
##########################################################################################
# Joint Fossil Birth-Death Process prior on the topology and fossil occurrence times #
##########################################################################################

# Define exponential priors on the birth rate and death rate #
# speciation rate
lambda <- 1.000001

# extinction rate
mu <- 1.0

# sampling fraction (in the present)
rho <- 0.00001

# The BD is conditioned on a starting time for the process, which is the origin time #
# Specify a uniform prior on the origin 
origin_time ~ dnUniform(300,750)

moves.append(mvScale(origin_time, weight=1.0, lambda=0.1))

In [3]:
### Define the tree-prior distribution as the birth-death process ###
timetree ~ dnBDP(rootAge=origin_time, lambda=lambda, mu=mu, rho=rho, taxa=taxa, samplingStrategy="uniform", condition="nTaxa")

In [4]:
timetree

  
   (((Hariotta_raleighana[&index=23]:225.180554,Struthio_camelus[&index=22]:225.180554)[&index=24]:68.164769,((Protopterus_aethiopicus[&index=21]:16.855105,Xenopus_tropicalis[&index=20]:16.855105)[&index=25]:210.436149,(Ornithorhynchus_anatinus[&index=19]:148.629698,Tetraodon_nigroviridis[&index=18]:148.629698)[&index=26]:78.661556)[&index=27]:66.054069)[&index=28]:283.944154,((((((Homo_sapiens[&index=17]:94.753713,Squalus_acanthias[&index=16]:94.753713)[&index=29]:227.958746,(Rhinochimaera_pacifica[&index=15]:33.464059,Polymixia_japonica[&index=14]:33.464059)[&index=30]:289.248400)[&index=31]:28.153888,Chimaera_sp[&index=13]:350.866347)[&index=32]:97.373732,((Callorhinchus_milii[&index=12]:156.869772,Hydrolagus_lemures[&index=11]:156.869772)[&index=33]:4.084804,Callorhinchus_capensis[&index=10]:160.954575)[&index=34]:287.285504)[&index=35]:11.560945,(Okamejei_kenojei[&index=9]:434.237286,(Heterodontus_francisci[&index=8]:410.866272,Chimaera_monstrosa[&index=7]:410.866272)[&index=36

In [5]:
# These moves update the node ages and the tree shape #
moves.append( mvNodeTimeSlideUniform(timetree, weight=20.0) )
moves.append( mvRootTimeSlideUniform(timetree, origin_time, weight=4.0) )
moves.append(mvSubtreeScale(timetree, weight=4))

In [6]:
# read tree topology from file
tree <- readTrees("../Projet/fishes/vertebrate_Poisson_NJ.tree", treetype="clock")[1]

   Attempting to read the contents of file "vertebrate_Poisson_NJ.tree"
   Successfully read file


In [7]:
tree

  
   ((((Squalus_acanthias[&index=23]:0.040710,((Mustelus_manazo[&index=22]:0.031320,Scyliorhinus_canicula[&index=21]:0.043080)[&index=24]:0.010170,Heterodontus_francisci[&index=20]:0.042350)[&index=25]:0.004480)[&index=26]:0.017110,((Amblyraja_radiata[&index=19]:0.021970,Okamejei_kenojei[&index=18]:0.031320)[&index=27]:0.036820,Plesiobatis_daviesi[&index=17]:0.084620)[&index=28]:0.009850)[&index=29]:0.023300,((((Chimaera_monstrosa[&index=16]:0.016590,Chimaera_sp[&index=15]:0.017640)[&index=30]:0.012410,Hydrolagus_lemures[&index=14]:0.033440)[&index=31]:0.011530,(Rhinochimaera_pacifica[&index=13]:0.020300,Hariotta_raleighana[&index=12]:0.023250)[&index=32]:0.026530)[&index=33]:0.009060,(Callorhinchus_callorynchus[&index=11]:0.003400,(Callorhinchus_capensis[&index=10]:0.002350,Callorhinchus_milii[&index=9]:0.001360)[&index=34]:0.000780)[&index=35]:0.046730)[&index=36]:0.074110)[&index=37]:0.011030,(((Tetraodon_nigroviridis[&index=8]:0.064100,Polymixia_japonica[&index=7]:0.056080)[&inde

In [8]:
# ajust terminal branch lengths so as to make the tree ultrametric
tree.makeUltrametric()

# rescale all the nodes of the tree
tree.rescale(750/tree.rootAge())

In [9]:
tree

  
   ((((Squalus_acanthias[&index=23]:606.856634,((Mustelus_manazo[&index=22]:566.089715,Scyliorhinus_canicula[&index=21]:566.089715)[&index=24]:28.300312,Heterodontus_francisci[&index=20]:594.390027)[&index=25]:12.466607)[&index=26]:47.612422,((Amblyraja_radiata[&index=19]:524.599288,Okamejei_kenojei[&index=18]:524.599288)[&index=27]:102.459929,Plesiobatis_daviesi[&index=17]:627.059216)[&index=28]:27.409840)[&index=29]:64.837489,((((Chimaera_monstrosa[&index=16]:421.248887,Chimaera_sp[&index=15]:421.248887)[&index=30]:34.533615,Hydrolagus_lemures[&index=14]:455.782502)[&index=31]:32.084817,(Rhinochimaera_pacifica[&index=13]:414.041630,Hariotta_raleighana[&index=12]:414.041630)[&index=32]:73.825690)[&index=33]:25.211487,(Callorhinchus_callorynchus[&index=11]:383.042075,(Callorhinchus_capensis[&index=10]:380.871549,Callorhinchus_milii[&index=9]:380.871549)[&index=34]:2.170525)[&index=35]:130.036732)[&index=36]:206.227738)[&index=37]:30.693455,(((Tetraodon_nigroviridis[&index=8]:640.499

In [10]:
# set the value of the tree topology
timetree.setValue(tree)

In [11]:
# node calibration
clade_1 <- clade("Cyprinus_carpio","Chimaera_monstrosa")
minimum_bound_clade_1 <- 422
maximum_bound_clade_1 <- 463
speciation_clade_1 := tmrca(timetree, clade_1)
obs_age_clade_1 ~ dnSoftBoundUniformNormal(min=minimum_bound_clade_1, max=maximum_bound_clade_1, sd=2, p=0.05)
obs_age_clade_1.clamp( (minimum_bound_clade_1 + maximum_bound_clade_1)/2 )

clade_2 <- clade("Cyprinus_carpio","Homo_sapiens")
minimum_bound_clade_2 <- 416
maximum_bound_clade_2 <- 422
speciation_clade_2 := tmrca(timetree, clade_2)
obs_age_clade_2 ~ dnSoftBoundUniformNormal(min=minimum_bound_clade_2, max=maximum_bound_clade_2, sd=2, p=0.05)
obs_age_clade_2.clamp( (minimum_bound_clade_2 + maximum_bound_clade_2)/2 )

clade_3 <- clade("Cyprinus_carpio","Tetraodon_nigroviridis")
minimum_bound_3 <- 150
speciation_clade_3 := tmrca(timetree, clade_3)
obs_age_clade_3 ~ dnExponential(0.2, offset = -speciation_clade_3)
obs_age_clade_3.clamp( -minimum_bound_3 )

clade_6 <- clade("Xenopus_tropicalis","Homo_sapiens")
minimum_bound_clade_6 <- 330
maximum_bound_clade_6 <- 350
speciation_clade_6 := tmrca(timetree, clade_6)
obs_age_clade_6 ~ dnSoftBoundUniformNormal(min=minimum_bound_clade_6, max=maximum_bound_clade_6, sd=2, p=0.05)
obs_age_clade_6.clamp( (minimum_bound_clade_6 + maximum_bound_clade_6)/2 )

clade_7 <- clade("Struthio_camelus","Homo_sapiens")
minimum_bound_clade_7 <- 312
maximum_bound_clade_7 <- 330
speciation_clade_7 := tmrca(timetree, clade_7)
obs_age_clade_7 ~ dnSoftBoundUniformNormal(min=minimum_bound_clade_7, max=maximum_bound_clade_7, sd=2, p=0.05)
obs_age_clade_7.clamp( (minimum_bound_clade_7 + maximum_bound_clade_7)/2 )

clade_8 <- clade("Ornithorhynchus_anatinus","Homo_sapiens")
minimum_bound_clade_8 <- 163
maximum_bound_clade_8 <- 191
speciation_clade_8 := tmrca(timetree, clade_8)
obs_age_clade_8 ~ dnSoftBoundUniformNormal(min=minimum_bound_clade_8, max=maximum_bound_clade_8, sd=2, p=0.05)
obs_age_clade_8.clamp( (minimum_bound_clade_8 + maximum_bound_clade_8)/2 )

clade_9 <- clade("Plesiobatis_daviesi","Chimaera_monstrosa")
minimum_bound_clade_9 <- 410
speciation_clade_9 := tmrca(timetree, clade_9)
obs_age_clade_9 ~ dnExponential(0.2, offset = -speciation_clade_9)
obs_age_clade_9.clamp( -minimum_bound_clade_9 )

clade_10 <- clade("Plesiobatis_daviesi","Mustelus_manazo")
minimum_bound_10 <- 190
speciation_clade_10 := tmrca(timetree, clade_10)
obs_age_clade_10 ~ dnExponential(0.2, offset = -speciation_clade_10)
obs_age_clade_10.clamp( -minimum_bound_10 )

clade_11 <- clade("Plesiobatis_daviesi","Okamejei_kenojei")
minimum_bound_11 <- 176
speciation_clade_11 := tmrca(timetree, clade_11)
obs_age_clade_11 ~ dnExponential(0.2, offset = -speciation_clade_11)
obs_age_clade_11.clamp( -minimum_bound_11 )

clade_14 <- clade("Heterodontus_francisci","Mustelus_manazo")
minimum_bound_14 <- 176
speciation_clade_14 := tmrca(timetree, clade_14)
obs_age_clade_14 ~ dnExponential(0.2, offset = -speciation_clade_14)
obs_age_clade_14.clamp( -minimum_bound_14 )

clade_15 <- clade("Scyliorhinus_canicula","Mustelus_manazo")
minimum_bound_15 <- 165
speciation_clade_15 := tmrca(timetree, clade_15)
obs_age_clade_15 ~ dnExponential(0.2, offset = -speciation_clade_15)
obs_age_clade_15.clamp( -minimum_bound_15 )

clade_16 <- clade("Callorhinchus_callorynchus","Chimaera_monstrosa")
minimum_bound_16 <- 161
speciation_clade_16 := tmrca(timetree, clade_16)
obs_age_clade_16 ~ dnExponential(0.2, offset = -speciation_clade_16)
obs_age_clade_16.clamp( -minimum_bound_16 )

clade_19 <- clade("Rhinochimaera_pacifica","Chimaera_monstrosa")
minimum_bound_19 <- 84
speciation_clade_19 := tmrca(timetree, clade_19)
obs_age_clade_19 ~ dnExponential(0.2, offset = -speciation_clade_19)
obs_age_clade_19.clamp( -minimum_bound_19 )


In [12]:
###########################################
# Substitution model #
###########################################
#Create the Q matrix. These data are amino acid sequences, so we initialize the MtRev matrix #
Q := fnMtRev()

# We assume a relaxed morphological clock rate, drawn from an exponential prior #
mean_clockrate ~ dnExponential(1.0)
moves.append(mvScale(mean_clockrate, weight=1.0))

relvar_clockrate ~ dnExponential(1.0)
moves.append(mvScale(relvar_clockrate, weight=1.0))

alpha := 1.0 / relvar_clockrate
beta := alpha / mean_clockrate

for (i in 1:n_branches) {
    clockrate[i] ~ dnGamma(alpha, beta)
}

# Specify moves on the clockrate #
for (i in 1:n_branches){
    moves.append(mvScale(clockrate[i], weight=1.0, lambda=0.1))
}

In [13]:
### Create the substitution model and clamp with our observed Standard data ###
seq ~ dnPhyloCTMC( tree=timetree, Q=Q, branchRates=clockrate,  type="AA")
seq.clamp( data )

In [14]:
data

   
   Protein character matrix with 23 taxa and 3248 characters
   Origination:                   aln_prot.fst
   Number of taxa:                23
   Number of included taxa:       23
   Number of characters:          3248
   Number of included characters: 3248
   Datatype:                      Protein
   
   


In [15]:
########
# MCMC #
########

# initialize the model object #
mymodel = model(timetree)

In [16]:
# monitors = VectorMonitors()

In [17]:
# Create a vector of monitors #
# 1. for the full model #
monitors[1] = mnModel(filename="analyses/fishes_relaxedclock.log", printgen=10, separator = TAB)

In [18]:
# 2. the tree #
monitors[2] = mnFile(timetree, filename="analyses/fishes_relaxedclock.trees", printgen=10, separator = TAB)

In [19]:
# 3. and a few select parameters to be printed to the screen #
monitors[3] = mnScreen(printgen=100, mean_clockrate, relvar_clockrate, origin_time)

In [ ]:
# Initialize the MCMC object #
mymcmc = mcmc(mymodel, monitors, moves)

# Run the MCMC #
mymcmc.run(generations=50000)

   
   Running MCMC simulation
   This simulation runs 1 independent replicate.
   The simulator uses 50 different moves in a random move schedule with 75 moves per iteration
   

Iter        |      Posterior   |     Likelihood   |          Prior   |   mean_clock..   |    origin_time   |   relvar_clo..   |    elapsed   |        ETA   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
0           |        -208461   |        -208234   |       -226.977   |      0.8850133   |            750   |       1.962005   |   00:00:00   |   --:--:--   |
100         |        -132284   |        -132087   |       -197.424   |      0.4829016   |       591.3626   |       1.465287   |   00:00:35   |   --:--:--   |
200         |        -109415   |        -109238   |       -176.954   |      0.5139251   |       560.7286   |       1.898347   |   00:01:11   |   04:54:39   |
300         |       -97213.9  

In [21]:
treetrace = readTreeTrace("analyses/fishes_relaxedclock.trees", treetype="clock", burnin=0.1)
map_tree = mccTree(treetrace, "analyses/fishes_relaxedclock.tree")


Restarting kernel...
Done!
